# Maneuver Test: Descent from trim and ground effect

The purpose of this Notebook is to illustrate how to simulate a descending trajectory in the easiest way, and how ground effect influences a light aircraft motion at very low altitudes above ground level.

## Setup

Let's start by setting up the simulation environment.

In [1]:
import os
import jsbsim
import sys
sys.path.insert(0, './utils/')

import plotting_routines as plrt
import data_handling as dh
from JSBSim_utils import CheckXMLFile

# Relative path to the directory where the flight model is stored
# Note - Aircraft directory needs to be writeable in order to modify the cg
PATH_TO_JSBSIM_FILES="../.."

print(f'=================================================================')
print(f'Current working directory: {os.getcwd()}')

# Global variables that must be modified to match your particular need
# The aircraft name
# Note - It should match the exact spelling of the model file
AIRCRAFT_NAME="c172x"

# The path to the input file c172c.xml
fdm_input_file_path = os.path.join(PATH_TO_JSBSIM_FILES, "aircraft", AIRCRAFT_NAME, f"{AIRCRAFT_NAME}.xml")

print(f'Input file path: {fdm_input_file_path}')

chk_fdm_file = CheckXMLFile(fdm_input_file_path, header='fdm_config')

# printe the check result
print(f'FDM input file check success: {chk_fdm_file}') 

# Console with log messages
jsbsim.FGJSBBase().debug_lvl = 1

# Instantiate the FDMExec object and load the aircraft
fdm = jsbsim.FGFDMExec(PATH_TO_JSBSIM_FILES)

load_status = fdm.load_model(AIRCRAFT_NAME)

# Set the output path to the current directory
fdm.set_output_path('examples/python/')

print(f'\nAircraft FDM: {AIRCRAFT_NAME}\nLoaded: {load_status}')
print(fdm)
print(f'=================================================================')

Current working directory: f:\agodemar\jsbsim\examples\python
Input file path: ../..\aircraft\c172x\c172x.xml
FDM input file check success: True

Aircraft FDM: c172x
Loaded: True
FGFDMExec 
root dir	:	../..
aircraft path	:	../../aircraft
engine path	:	../../engine
systems path	:	../../systems
output path	:	../../examples/python



## Explore the FDM catalog of properties

In [2]:
# Get the aircraft catalog of properties
# Part of them is defined internally by JSBSim, and part of them is defined in the FDM input file
catalog = fdm.get_property_catalog()
# catalog is a list of strings, print the entries
for entry in catalog:
    print(entry)

inertial/sea-level-radius_ft (R)
simulation/gravity-model (RW)
simulation/integrator/rate/rotational (RW)
simulation/integrator/rate/translational (RW)
simulation/integrator/position/rotational (RW)
simulation/integrator/position/translational (RW)
simulation/write-state-file (W)
simulation/channel-dt (R)
simulation/gravitational-torque (RW)
simulation/force-output (W)
simulation/do_simple_trim (W)
simulation/do_linearization (W)
simulation/reset (W)
simulation/disperse (R)
simulation/randomseed (RW)
simulation/terminate (RW)
simulation/pause (RW)
simulation/sim-time-sec (R)
simulation/dt (R)
simulation/jsbsim-debug (RW)
simulation/frame (RW)
simulation/trim-completed (RW)
simulation/output/log_rate_hz (RW)
simulation/output/enabled (RW)
velocities/h-dot-fps (R)
velocities/v-north-fps (R)
velocities/v-east-fps (R)
velocities/v-down-fps (R)
velocities/u-fps (R)
velocities/v-fps (R)
velocities/w-fps (R)
velocities/p-rad_sec (R)
velocities/q-rad_sec (R)
velocities/r-rad_sec (R)
velocities

## Sim initialization

In [3]:
# --- Simulation Initialization ---

# Initial conditions
fdm['ic/h-sl-ft']       = 100.0  # altitude above sea level (ft)
fdm['ic/vc-kts']        =  40.0  # calibrated airspeed (kts)
fdm['ic/latitude-deg']  =  40.0  # latitude (deg)
fdm['ic/longitude-deg'] = 14.2   # longitude (deg
fdm['ic/psi-deg']       =  90.0  # heading (deg)
fdm['ic/phi-deg']       =   0.0  # roll angle (deg)
fdm['ic/gamma-deg']     =   0.0  # flight path angle (deg)
fdm['ic/running']       = -1     # set the engine running

ic_status = fdm.run_ic()
print(f'Initial conditions set: {ic_status}')

Initial conditions set: True


## Data storage

In [4]:
# fdm.enable_output()
# print(fdm.get_root_dir())
# output_directive_fcs = 'output_fcs.xml'
# fdm.get_output_filename(1)
# -- No output files --
fdm.disable_output()

# --- Data Storage ---
# This section initializes lists to record simulation data.

times = []      # List to record the simulation time at each step.
betas = []      # List to record the beta angle at each step.
bankAngle = []  # List to record the bank angle at each step.
ailerons = []   # List to record the aileron control surface deflection.
rudder = []     # List to record the rudder control surface deflection.


## Simulation

### Initial trim

In [5]:
# Attempt to trim the aircraft.
try:
    # 1 means straight flight by using all changeable control variables.
    fdm['simulation/do_simple_trim'] = 1

except jsbsim.TrimFailureError:
    print("Trim failed, continuing rudder kick in an untrimmed state.")
    fdm.run_ic()
    pass  # Ignore trim failure

Trim failed, continuing rudder kick in an untrimmed state.


### Simulation loop

In [9]:
fg_propagate = fdm.get_propagate()
print(f"UVW:\n {fg_propagate.get_uvw()}")
print(f"Tec2b:\n {fg_propagate.get_Tec2b()}")
print(f"Tl2b:\n {fg_propagate.get_Tl2b()}") 
# fg_propagate.get:_state() # not defined


UVW:
 [[67.61116492]
 [ 0.        ]
 [ 0.        ]]
Tec2b:
 [[ 2.22044605e-16  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-1.00000000e+00  0.00000000e+00  2.22044605e-16]]
Tl2b:
 [[ 1.00000000e+00  0.00000000e+00 -2.22044605e-16]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 2.22044605e-16  0.00000000e+00  1.00000000e+00]]
